<a href="https://colab.research.google.com/github/ipeirotis/dealing_with_data/blob/master/04-Web_Scraping/B-Crawling_HTML_Pages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Crawling and Extracting Data from Websites

In [ ]:
# !sudo -H apt-get -y install libxml2-dev libxslt-dev python3-dev
# !sudo -H pip3 install -U lxml

## Searching in HTML: Fetching the webpage title from ESPN.com

Let's start by trying to fetch the headlines from the site ESPN.com.



In [ ]:
import requests # This command allows us to fetch URLs
from lxml import html # This module will allow us to parse the returned HTML/XML
import pandas # To create a dataframe

# Let's start by fetching the page, and parsing it
url = "http://www.espn.com/"
response = requests.get(url) # get the html of that url
doc = html.fromstring(response.text) # parse it and create a document

In [ ]:
doc

Let's start by getting the content of the `<title>` node from the site:

In [ ]:
# This will return back a list of nodes that match
# are called <title>......</title>
results = doc.xpath('//title/text()')
results

In [ ]:
results[0]

In [ ]:
results = doc.xpath('//title')
results

In [ ]:
# gives only the text stored directly under the node
results[0].text 

In [ ]:
# gives all the text stored directly under the node *and* its children
# for the <title> node, it does not make a difference
results[0].text_content() 

In [ ]:
# Compare the two results to see the difference
# The first one gives all the text stored under the root <HTML> node
# The second gives only the text immediately under HTML (but not under the children)
# doc.xpath("/html")[0].text_content()
# doc.xpath("/html")[0].text

### Exercise

* Connect to the NYU Stern website, and fetch the title of the page

In [ ]:
# your code here

#### Solution

In [ ]:
stern_url = 'http://www.stern.nyu.edu'
stern_html = requests.get(stern_url).text
stern_doc = html.fromstring(stern_html)

In [ ]:
title1 = stern_doc.xpath("//title/text()")
title1[0]

In [ ]:
title2 = stern_doc.xpath("//title")
title2[0].text

## Searching for elements of interest in the web page

The `doc` variable is an `HtmlElement` object, and we can now use **XPath** queries to locate the elements that we need. (Depending on time, we may do in class a tutorial on XPath. For now, you can look at the [W3Schools tutorial](http://www.w3schools.com/xpath/xpath_nodes.asp))

For example, to find all the `<a ...> ... </a>` tags in the returned html, which store the links in the page, we issue the command:

In [ ]:
links = doc.xpath("//a")
len(links)

In [ ]:
# Let's pick now a random link

lnk = links[80]
type(lnk)

The `lnk` variable is again an HtmlElement. To get parts of the html element that we need, we can use the `get` method (e.g., to get the `href` attribute) and the `text` method (to get the text within the `<a>...</a>` tag.

In [ ]:
lnk.get("href")

In [ ]:
lnk.text_content()

In [ ]:
# The strip() removes blank spaces before and after the text
lnk.text_content().strip()

Let's put everything together

In [ ]:
links = doc.xpath("//a")
# Iterates over all the links (this means all the nodes
# that matched the //a XPath query) and prints the content
# of the attribute href and the text for that node
for link in links:
    print("==================================")
    print(link.get("href"), "==>", link.text_content().strip())

Now, let's revisit the _list comprehension_ approach that we discussed in the Python Primer session, for quickly constructing lists:

In [ ]:
urls = [lnk.get("href") for lnk in doc.xpath('//a')]
urls

### Exercise

Use a list compresension approach, to get the text_content of all the URLs in the page.

In [ ]:
# your code here


And now create a list where we put together text content and the URL for each link

In [ ]:
# your code here

#### Solution

In [ ]:
text = [lnk.text_content().strip() for lnk in doc.xpath("//a")]
text

In [ ]:
# Creating a list of tuples where we put together href and text for each link
list_tuples = [(lnk.get("href"), lnk.text_content().strip()) for lnk in doc.xpath("//a")]

In [ ]:
# Creating a list of dictionaries with the text and URL for each link
list_dicts = [{"URL": lnk.get("href"), "Text": lnk.text_content().strip()} for lnk in doc.xpath("//a")]

In [ ]:
import pandas as pd
pd.DataFrame(list_dicts)

### More Advanced Example: Get the list of headlines from ESPN


Now, let's examine how we can get the data from the website. The key is to understand the structure of the HTML, where the data that we need is stored, and how to fetch the elements. Then, using the appropriate XPath queries, we will get what we want.

Let's start by fetching the page, and parsing it

In [ ]:
url = "http://www.espn.com/"
response = requests.get(url) # get the html of that url
doc = html.fromstring(response.text) # parse it and create a document

By using the `"Right-Click > Inspect"` option of Chrome,
we right click on the headlines and select `"Inspect"`.
This opens the source code.
There we see that all under a `<div class="headlineStack">` tag.

In [ ]:
headlineNode = doc.xpath('//div[@class="headlineStack"]')

The result of that operation is a list with 8 elements.

In [ ]:
type(headlineNode)

In [ ]:
len(headlineNode)

Each headline is under a  `<li><a href="...."></a>` tag.
So, we get all the `<li><a ...>` tags within the `<div class="headlineStack">`
(which is stored in the "`headlineNode`" variable)

In [ ]:
headlines = headlineNode[0].xpath('.//li/a')
len(headlines)

Now, we have the nodes with the conent in the headlines variable.
We extract the text and the URL.

In [ ]:
data = [{"Title": a.text_content(), "URL": a.get("href")} for a in headlines]
data

And let's create our dataframe, so that we can have a better view

In [ ]:
dataframe = pandas.DataFrame(data)
dataframe

#### Of course, there are always more than one way to skin a cat...

Alternatively, if we did not want to restrict ourselves to just the first headline box, we could write an alternative query, to get back all the headlines, that appear in an XPath `//div[@class="headlineStack"]//li/a`:

In [ ]:
headlines = doc.xpath('//div[@class="headlineStack"]//li/a')
data = [{"Title": a.text_content(), "URL": a.get("href")} for a in headlines]
df = pandas.DataFrame(data)
df

In [ ]:
headlines = doc.xpath('//a[@data-mptype="headline"]')
data = [{"Title": a.text_content(), "URL": a.get("href")} for a in headlines]
df = pandas.DataFrame(data)
df

## In Class Example: Crawl BuzzFeed

* We will try to get the top articles that appear on Buzzfeed
* We will grab the link for the article, the text of the title, the description, and the editor.
* The results will be stored in a dataframe (we will see in detail what a dataframe is, in a couple of modules)


In [ ]:
#your code here
import requests

from lxml import html

resp = requests.get("http://www.buzzfeed.com")
doc = html.fromstring(resp.text)

In [ ]:
story_nodes = doc.xpath('//article[@data-buzzblock="story-card"]')

#### Solution for Buzzfeed (as of Septenber 22, 2022)

In [ ]:
import requests # This command allows us to fetch URLs
from lxml import html # This module will allow us to parse the returned HTML/XML
import pandas
import re

# Let's start by fetching the page, and parsing it
url = "http://www.buzzfeed.com/"
response = requests.get(url) # get the html of that url
doc = html.fromstring(response.text) # parse it and create a document

articleNodes = doc.xpath("//div[@class='feedItem_textWrapper__S6osO']") 

def parseArticleNode(article):
    headline = article.xpath(".//h2")[0].text_content().strip()
    headline_link = article.xpath(".//a")[0].get("href")

    editor_node = article.xpath(".//div[@class='xs-text-6 text-gray xs-mt1']")
    if len(editor_node)>0:
      editor = editor_node[0].text_content().strip()
    else:
      editor = ""
    regex = re.compile(r'^by (.*)$')
    matches = regex.finditer(editor)
    for m in matches:
      editor = m.group(1)


    result = {
        "headline": headline,
        "URL" : headline_link,
        "editor" : editor
    }
    return result

data = [parseArticleNode(article) for article in articleNodes]
df = pandas.DataFrame(data)
df